In [1]:
import sys
sys.path.append("./Recommenders/")
# chagne the working directory to https://www.johnny-lin.com/cdat_tips/tips_pylang/path.html
import pandas as pd
import pyspark

from reco_utils.common.spark_utils import start_or_get_spark
from reco_utils.evaluation.spark_evaluation import SparkRankingEvaluation, SparkRatingEvaluation
from reco_utils.recommender.sar.sar_singlenode import SARSingleNode
from reco_utils.dataset.url_utils import maybe_download
from reco_utils.dataset.python_splitters import python_random_split

In [2]:
print("System version: {}".format(sys.version))
print("Pandas version: {}".format(pd.__version__))
print("PySpark version: {}".format(pyspark.__version__))

System version: 3.6.8 |Anaconda, Inc.| (default, Dec 29 2018, 19:04:46) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
Pandas version: 0.24.1
PySpark version: 2.4.0


### load data

In [5]:
df_true = pd.read_csv('./fl_processed_test_split_03062019.csv', index_col=0)

In [13]:
df_true.columns = ['uid','iid','Rating']

### result from Mars and Alex

In [3]:
result = pd.read_csv('./result_0313.csv', index_col = 0)

/Users/yupinglin/anaconda3/envs/reco_pyspark/lib/python3.6/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [8]:
result.head()

,uid,iid,r_ui,est,details
0,0,3524,1672.0,5.000000,{'was_impossible': False}
1,0,1153,1671.0,3.842258,{'was_impossible': False}
2,0,3992,1670.0,4.688061,{'was_impossible': False}
3,0,3927,1669.0,4.155132,{'was_impossible': False}
4,0,3697,1668.0,3.761152,{'was_impossible': False}


In [7]:
df_pred_mars = result[['uid','iid','r_ui']]

In [9]:
df_pred_mne = result[['uid','iid','est']]

### define the header

In [18]:
COL_USER = 'UserId'
COL_ITEM = 'MovieId'
COL_RATING = 'Rating'
COL_PREDICTION = 'r_ui' 

HEADER = {
    "col_user": COL_USER,
    "col_item": COL_ITEM,
    "col_rating": COL_RATING,
    "col_prediction": COL_PREDICTION,
}

### pred df
 - turn into index

In [72]:
# df_pred = pd.read_csv('./result_0307.csv', index_col=0)
# df_pred = pd.read_csv('/Users/yupinglin/Desktop/ForwardLane/ALEX-ranking/data/clean_ranking_result_0307.csv', index_col=0)

In [75]:
# df_pred.head()

,rank_score,client_id,document_id
0,2.0,ATLSAC6A247DFCFD,Discovery Fund Commentary
1,2.0,HS00000004030108,Discovery Fund Commentary
2,2.0,ATLS5387E9B2267C,Discovery Fund Commentary
3,4.0,ATLS5387E9B2267C,Discovery Fund Commentary
4,2.0,HS00000001395771,Discovery Fund Commentary


### true df
 - turn into index

In [76]:
# df_true = pd.read_csv('./fl_processed_test_split_03062019.csv', index_col=0)
# df_true = pd.read_csv('/Users/yupinglin/Desktop/ForwardLane/ALEX-ranking/data/ranking_2.csv')

### Rating matric
-  the is `nan` since the prediciton is never show up in the ground true table

In [ ]:
spark = start_or_get_spark("EvaluationTesting", "local")

dfs_true = spark.createDataFrame(df_true)
dfs_pred = spark.createDataFrame(df_pred)

In [67]:
spark_rate_eval = SparkRatingEvaluation(dfs_true, dfs_pred, **HEADER)

In [68]:
spark_rate_eval.rmse()

nan

In [57]:
print("The RMSE is {}".format(spark_rate_eval.rmse()))
print("The R2 is {}".format(spark_rate_eval.rsquared()))
print("The MAE is {}".format(spark_rate_eval.mae()))
print("The explained variance is {}".format(spark_rate_eval.exp_var()))

The RMSE is nan
The R2 is nan
The MAE is nan
The explained variance is nan


### Ranking

In [87]:
spark_rank_eval = SparkRankingEvaluation(dfs_true, dfs_pred, k=50, relevancy_method="top_k", **HEADER)

top k: 5

***

### ranking for mars

In [35]:
COL_USER = 'uid'
COL_ITEM = 'iid'
COL_RATING = 'Rating'
COL_PREDICTION = 'r_ui' 

HEADER = {
    "col_user": COL_USER,
    "col_item": COL_ITEM,
    "col_rating": COL_RATING,
    "col_prediction": COL_PREDICTION,
}

In [16]:
spark = start_or_get_spark("EvaluationTesting", "local")

dfs_true = spark.createDataFrame(df_true)
dfs_pred = spark.createDataFrame(df_pred_mars)

In [19]:
spark_rank_eval = SparkRankingEvaluation(dfs_true, dfs_pred, k=10, relevancy_method="top_k", **HEADER)

In [20]:
print("The precision at k is {}".format(spark_rank_eval.precision_at_k()))
print("The recall at k is {}".format(spark_rank_eval.recall_at_k()))
print("The ndcg at k is {}".format(spark_rank_eval.ndcg_at_k()))
print("The map at k is {}".format(spark_rank_eval.map_at_k()))

The precision at k is 0.23778841967784062
The recall at k is 0.3746475734394758
The ndcg at k is 0.338815815750349
The map at k is 0.20963242015311745


In [27]:
spark_rank_eval = SparkRankingEvaluation(dfs_true, dfs_pred, k=5, relevancy_method="top_k", **HEADER)

In [28]:
print("The precision at k is {}".format(spark_rank_eval.precision_at_k()))
print("The recall at k is {}".format(spark_rank_eval.recall_at_k()))
print("The ndcg at k is {}".format(spark_rank_eval.ndcg_at_k()))
print("The map at k is {}".format(spark_rank_eval.map_at_k()))

The precision at k is 0.28062690465825
The recall at k is 0.22392751760405213
The ndcg at k is 0.29156365754124514
The map at k is 0.15165091688491777


In [36]:
spark_rank_eval = SparkRankingEvaluation(dfs_true, dfs_pred, k=20, relevancy_method="top_k", **HEADER)

In [37]:
print("The precision at k is {}".format(spark_rank_eval.precision_at_k()))
print("The recall at k is {}".format(spark_rank_eval.recall_at_k()))
print("The ndcg at k is {}".format(spark_rank_eval.ndcg_at_k()))
print("The map at k is {}".format(spark_rank_eval.map_at_k()))

The precision at k is 0.1712233347845015
The recall at k is 0.5268346912121399
The ndcg at k is 0.41172638220397156
The map at k is 0.24687015498401804


### ranking for NMF

In [32]:
COL_USER = 'uid'
COL_ITEM = 'iid'
COL_RATING = 'Rating'
COL_PREDICTION = 'est' 

HEADER = {
    "col_user": COL_USER,
    "col_item": COL_ITEM,
    "col_rating": COL_RATING,
    "col_prediction": COL_PREDICTION,
}

In [22]:
spark = start_or_get_spark("EvaluationTesting", "local")
dfs_pred_mne = spark.createDataFrame(df_pred_mne)

In [23]:
spark_rank_eval_mne = SparkRankingEvaluation(dfs_true, dfs_pred_mne, k=10, relevancy_method="top_k", **HEADER)

In [24]:
print("The precision at k is {}".format(spark_rank_eval_mne.precision_at_k()))
print("The recall at k is {}".format(spark_rank_eval_mne.recall_at_k()))
print("The ndcg at k is {}".format(spark_rank_eval_mne.ndcg_at_k()))
print("The map at k is {}".format(spark_rank_eval_mne.map_at_k()))

The precision at k is 0.13217239878101875
The recall at k is 0.2004153105154411
The ndcg at k is 0.20983389233741537
The map at k is 0.1173076550191498


In [30]:
spark_rank_eval_mne = SparkRankingEvaluation(dfs_true, dfs_pred_mne, k=5, relevancy_method="top_k", **HEADER)

In [31]:
print("The precision at k is {}".format(spark_rank_eval_mne.precision_at_k()))
print("The recall at k is {}".format(spark_rank_eval_mne.recall_at_k()))
print("The ndcg at k is {}".format(spark_rank_eval_mne.ndcg_at_k()))
print("The map at k is {}".format(spark_rank_eval_mne.map_at_k()))

The precision at k is 0.1887679582063561
The recall at k is 0.14455587343049256
The ndcg at k is 0.20976641591907902
The map at k is 0.09943582957296496


In [33]:
spark_rank_eval_mne = SparkRankingEvaluation(dfs_true, dfs_pred_mne, k=20, relevancy_method="top_k", **HEADER)

In [34]:
print("The precision at k is {}".format(spark_rank_eval_mne.precision_at_k()))
print("The recall at k is {}".format(spark_rank_eval_mne.recall_at_k()))
print("The ndcg at k is {}".format(spark_rank_eval_mne.ndcg_at_k()))
print("The map at k is {}".format(spark_rank_eval_mne.map_at_k()))

The precision at k is 0.08354375272094036
The recall at k is 0.2522474103558126
The ndcg at k is 0.23443931404597732
The map at k is 0.12670985953617006
